In [ ]:
# Install what needs to be installed setting up instance here
!pip uninstall -y bitsandbytes accelerate

%pip install -U bitsandbytes accelerate
import bitsandbytes as bnb
import accelerate
import csv

# Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#enabling import of custom module for colab
! cp /content/gdrive/MyDrive/Util.py . 

In [ ]:
# Set up file location here
import sys
sys.path.append('/content/gdrive/MyDrive')

# Import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
import json
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
#load the data
general_df, prompt_g, correct_g, gtp4_g, Info_g = load_data_JSON('/content/gdrive/MyDrive/general_data.json', 'general')
qa_df, prompt_q, correct_q, gtp4_q, Info_q = load_data_JSON('/content/gdrive/MyDrive/qa_data.json', 'qa')
sum_df, prompt_s, correct_s, gtp4_s, Info_s = load_data_JSON('/content/gdrive/MyDrive/summarization_data.json', 'sum')

In [ ]:
# Access input (API key, etc.)
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf", access)

In [ ]:
# Text generation pipeline setup
gen = pipeline("text-generation", model=model, torch_dtype=torch.float16, tokenizer=tokenizer)

In [ ]:
# Function to generate responses
def get_response(prompt, max_len):
    sequences = gen(prompt, do_sample=True, top_k=5, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=max_len + len(prompt))
    return sequences[0]['generated_text']

In [ ]:
# ChainPoll method to detect hallucinations
def chainpoll(model, tokenizer, prompts, num_responses=5):
    out_q = []

    # Process the questions (currently the first 2 questions)
    for i, prompt in enumerate(prompts[:2]):
        hallucination_scores = []

        # Run 5 times for the ChainPoll method
        for _ in range(num_responses):
            # Tokenize the prompt for input
            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

            # Generate the response from the model
            with torch.no_grad():
                outputs = model.generate(
                    inputs['input_ids'],
                    max_length=50,  # Adjust based on needs
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the output response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Ask the model if there are hallucinations using the CoT method
            hallucination_prompt = f"Does the following output contain hallucinations? Explain in detail:\n\nOutput: {response}\n"
            hallucination_inputs = tokenizer(hallucination_prompt, return_tensors="pt").to("cuda")

            # Generate the hallucination response
            with torch.no_grad():
                hall_outputs = model.generate(
                    hallucination_inputs['input_ids'],
                    max_length=100,  # Adjust this as needed to allow space for CoT
                    num_beams=5,
                    early_stopping=True
                )

            # Decode the hallucination response
            hallucination_response = tokenizer.decode(hall_outputs[0], skip_special_tokens=True)

            # Check for "yes" or "no" in the hallucination response and store the score
            if "yes" in hallucination_response.lower():
                hallucination_scores.append(0)
            else:
                hallucination_scores.append(1)

        # Compute final hallucination score as the proportion of "yes" responses
        hallucination_score = sum(hallucination_scores) / num_responses

        # Save the final hallucination judgment and explanation
        out_q.append({
            "prompt": prompt,
            "response": response,
            "hallucination_score": hallucination_score,
            "explanation": hallucination_response
        })

    return out_q

In [ ]:
chainpoll_results = chainpoll(model, tokenizer, prompt_q)

In [ ]:
# Process and format the ChainPoll results
out_q = []
for result in chainpoll_results:
    prompt = result["prompt"]
    response = result["response"]
    hallucination_score = result["hallucination_score"]
    explanation = result["explanation"]

    # Create a formatted string for each result
    formatted_output = f"""
    Prompt: {prompt}

    Response: {response}

    Hallucination Score: {hallucination_score}

    Explanation: {explanation}
    """
    
    # Append results to out_q for writing
    out_q.append(formatted_output)

# Optionally print out the formatted results for checking
# for formatted_result in out_q:
#    print(formatted_result)

In [ ]:
# Write the results
def write_out(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',')  # Fix delimiter to a single character
        csv_writer.writerows([elt] for elt in data)

write_out("/content/gdrive/MyDrive/ChainPoll_qa_results.csv", out_q)